In [22]:
import pandas as pd
import math
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib
import matplotlib.pyplot as plt             #visualisation
import os
from sklearn import preprocessing 

%matplotlib inline     
sns.set(color_codes=True)

RQ3: Does the distance of an Airbnb property from the city’s 5 highest rated attractions (as per trip advisor) affect the likelihood of it being vacant?

In [23]:
path = './../London_data/'

listings = pd.read_csv(os.path.join(path, "listings.csv"))
listings.head(10)

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,198258,https://www.airbnb.com/rooms/198258,20231210055232,2023-12-10,city scrape,Rental unit in Barking · ★4.74 · 1 bedroom · 1...,NaN,"I live in Barking town centre, at one time the...",https://a0.muscache.com/pictures/airflow/Hosti...,967537,...,4.88,4.45,4.68,NaN,f,1,0,1,0,0.27
1,33332,https://www.airbnb.com/rooms/33332,20231210055232,2023-12-10,city scrape,"Home in St Margaret's, Isleworth · ★4.40 · 1 ...",NaN,Peaceful and friendly.,https://a0.muscache.com/pictures/miso/Hosting-...,144444,...,4.53,4.68,4.26,NaN,f,2,0,2,0,0.12
2,42010,https://www.airbnb.com/rooms/42010,20231210055232,2023-12-10,city scrape,Home in East Finchley · ★4.88 · 1 bedroom · 1 ...,NaN,We have a unique cinema called the Phoenix whi...,https://a0.muscache.com/pictures/airflow/Hosti...,157884,...,4.95,4.73,4.87,NaN,t,2,0,2,0,3.45
3,284603,https://www.airbnb.com/rooms/284603,20231210055232,2023-12-11,previous scrape,Rental unit in London · ★4.83 · 1 bedroom · 1 ...,NaN,NaN,https://a0.muscache.com/pictures/3246231/8e9b1...,1481851,...,5.00,5.00,4.83,NaN,f,1,1,0,0,0.04
4,89870,https://www.airbnb.com/rooms/89870,20231210055232,2023-12-10,city scrape,Rental unit in London · ★4.65 · 1 bedroom · 1 ...,NaN,Finsbury Park is a friendly melting pot commun...,https://a0.muscache.com/pictures/miso/Hosting-...,54730,...,4.91,4.73,4.50,NaN,f,3,2,1,0,0.89
5,326146,https://www.airbnb.com/rooms/326146,20231210055232,2023-12-13,previous scrape,Rental unit in London · 1 bedroom · 1 bed,NaN,NaN,https://a0.muscache.com/pictures/3536712/9c7a9...,1667975,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
6,96052,https://www.airbnb.com/rooms/96052,20231210055232,2023-12-10,city scrape,Condo in London · ★4.82 · 2 bedrooms · 1 bed ·...,NaN,"Residential, quiet and green neighbourhood wit...",https://a0.muscache.com/pictures/airflow/Hosti...,448154,...,4.90,4.45,4.70,NaN,f,2,0,2,0,0.52
7,381467,https://www.airbnb.com/rooms/381467,20231210055232,2023-12-11,previous scrape,Rental unit in London · ★4.66 · 2 bedrooms · 2...,NaN,Peaceful and quite with beautiful tree lined s...,https://a0.muscache.com/pictures/4700556/fcce1...,1913410,...,4.82,4.88,4.64,NaN,f,1,1,0,0,1.86
8,390987,https://www.airbnb.com/rooms/390987,20231210055232,2023-12-10,city scrape,Home in London · ★4.73 · 8 bedrooms · 1 bed · ...,NaN,The neighbourhood is safe and many new cafes a...,https://a0.muscache.com/pictures/8841768/8130b...,1955537,...,4.83,4.75,4.75,NaN,f,3,0,3,0,0.09
9,394755,https://www.airbnb.com/rooms/394755,20231210055232,2023-12-11,previous scrape,Rental unit in London · 2 bedrooms · 2 beds · ...,NaN,NaN,https://a0.muscache.com/pictures/2b043814-44d7...,1973515,...,5.00,5.00,4.00,NaN,f,1,1,0,0,0.01


According to TripAdvisor, these are the top 5 highest rated attractions in London and their geographic position (https://www.tripadvisor.ca/Attractions-g186338-Activities-London_England.html):
1. Tower of London - (51.508530, -0.076132)
2. The British Museum - (51.518757, -0.126168)
3. London Eye - (51.503399, -0.119519)
4. National Gallery - (51.508972, -0.128794)
5. Tower Bridge - (51.505554, -0.075278)

In [26]:
# Here is the list of attractions in London and their coordinates
attract_coords = [("Tower_of_London", (51.508530, -0.076132)),("British_Museum", (51.518757, -0.126168)),("London_Eye", (51.503399, -0.119519)),("National_Gallery", (51.508972, -0.128794)),("Tower_Bridge", (51.505554, -0.075278))]

In [27]:
EARTH_RADIUS_KM = 6371

def to_radians(degrees):
    return degrees * math.pi / 180

def get_distance(lat1, lon1, lat2, lon2):
    r_lat1 = to_radians(lat1)
    r_lat2 = to_radians(lat2)
    d_lat = to_radians(lat2 - lat1)
    d_lon = to_radians(lon2 - lon1)
    a = ((math.sin(d_lat / 2) ** 2) +
         (math.cos(r_lat1) * math.cos(r_lat2) * (math.sin(d_lon / 2) ** 2)))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return EARTH_RADIUS_KM * c

Feature Engineering

In [28]:
# Create new features to measure the distance between each property listing and each tourist attraction
for attract_name, attract_coord in attract_coords:
    # Calculate distance for each attraction
    listings[attract_name + '_Distance'] = listings.apply(
        lambda row: get_distance(row['latitude'], row['longitude'], attract_coord[0], attract_coord[1]),
        axis=1
    )

                       id                                      listing_url  \
0                  198258              https://www.airbnb.com/rooms/198258   
1                   33332               https://www.airbnb.com/rooms/33332   
2                   42010               https://www.airbnb.com/rooms/42010   
3                  284603              https://www.airbnb.com/rooms/284603   
4                   89870               https://www.airbnb.com/rooms/89870   
...                   ...                                              ...   
91773  950589815013504257  https://www.airbnb.com/rooms/950589815013504257   
91774  951178246905249947  https://www.airbnb.com/rooms/951178246905249947   
91775  951188392382129035  https://www.airbnb.com/rooms/951188392382129035   
91776  951192793768996976  https://www.airbnb.com/rooms/951192793768996976   
91777  952607914901368427  https://www.airbnb.com/rooms/952607914901368427   

            scrape_id last_scraped           source  \
0      2